# Лабораторная работа №3

## Выполнил студент группы БВТ1905 Максимов Данила Васильевич

### Задача №1

Реализовать методы поиска подстроки в строке. Добавить возможность
ввода строки и подстроки с клавиатуры. Предусмотреть возможность
существования пробела. Реализовать возможность выбора опции
чувствительности или нечувствительности к регистру. Оценить время работы
каждого алгоритма поиска и сравнить его со временем работы стандартной
функции поиска, используемой в выбранном языке программирования.


#### Алгоритм Кнута-Морриса-Пратта (1)

In [3]:
import time
#функция заполнения массива пи наибольшими длинами суффиксов, совпадающих с соответств. префиксами для текущего элемента
def syf_max_len(s):
    pi = [0]*len(s)
    j = 0
    i = 1
    while i < len(s):
        if s[j] == s[i]:
            pi[i] = j +1
            i += 1
            j += 1
        else:
            if j == 0:
                pi[i] = 0
                i += 1
            else:
                j = pi[j - 1]
    return pi

def kmp_search(string, substring):
    pi = syf_max_len(string)
    n = len(string)
    m = len(substring)
    i = j = 0
    while i < n:
        #пока равны элементы перебираем посимвольно 
        if string[i] == substring[j]:
            i += 1
            j += 1
            #конец построки - найдено
            if j == m:
                return True
        #когда находится несовпадение берем предыдущий элемент из массива пи и смещаем
        #указатель подстроку на позицию первого несовпавшего элемента после цепочки совпавших
        else:
            if j > 0:
                j = pi[j - 1]
            else:
                i += 1
    return False

def search():
    string = input("Введите строку: ")
    substring = input("Введите подстроку: ")
    register = input("Чувствительность к регистру? Введите да/нет: ")
    if register == "да":
        return kmp_search(string, substring)
    elif register == "нет":
        return kmp_search(string.lower(),substring.lower())
start = time.time()
print(search())
print("Время работы алгоритма поиска Кнута-Морриса-Пратта:", (time.time() - start) * 1000, "миллисекунд")

Введите строку: парабола параплан ПАраплан параплАН
Введите подстроку: параплан 
Чувствительность к регистру? Введите да/нет: нет
True
Время работы алгоритма поиска Кнута-Морриса-Пратта: 10754.033327102661 миллисекунд


#### Упрощенный Бойера-Мура (2)

In [5]:
def offset_table(substring):
    lenght = len(substring)
    glossary = set()
    offset = {}
    #цикл с предпоследнего до первого элемента
    for i in range(lenght - 2, -1, -1):
        if substring not in glossary:
            #определяем смещение от конца образа
            offset[substring[i]] = lenght - i - 1
            #заносим символ в таблицу
            glossary.add(substring[i])
    #отдельное смещение для последнего символа
    if substring[lenght - 1] not in glossary:
        offset[substring[lenght - 1]] = lenght
    #определяем смещение для всех символов, которых нет в этой подстроке
    offset['#'] = lenght
    return offset

def bm_search(string, substring):
    offset = offset_table(substring)
    n = len(string)
    m = len(substring)
    bias = 0
    if n >= m:
        #i - указатель на символ в строке, j - в подстроке
        i = m - 1
        while i < n:
            #счетчик, чтобы пробежаться по всем символам между 0 и i
            k = 0
            j = 0
            flag = False
            for j in range(m-1, -1, -1):
                #сравниваем попарно символы
                if string[i-k] != substring[j]:
                    #если символ последний, то определяем смещение по символу строки
                    if j == m-1:
                        #если есть в словаре, то на его смещение
                        if offset.get(string[i]) == True:
                            bias = offset[string[i]]
                        #если нет - для прочих
                        else:
                            bias = offset['#']
                    #если не последний, то смещение по символу подстроки
                    else:
                        bias = offset[substring[j]]
                    #смещаем указатель строки и прерываем
                    i += bias
                    break
                k += 1
            #если дошли до начала образа, значит все его символы совпали
            if not flag:
                return True
    else:
        return False
    

def search():
    string = input("Введите строку: ")
    substring = input("Введите подстроку: ")
    register = input("Чувствительность к регистру? Введите да/нет: ")
    if register == "да":
        return bm_search(string, substring)
    elif register == "нет":
        return bm_search(string.lower(),substring.lower())
start = time.time()
print(search())
print("Время работы алгоритма поиска Бойера-Мура:", (time.time() - start) * 1000, "миллисекунд")

Введите строку: парабола параплан ПАраплан параплАН
Введите подстроку: параплан 
Чувствительность к регистру? Введите да/нет: да
True
Время работы алгоритма поиска Бойера-Мура: 8158.447742462158 миллисекунд


### Задача №2

Игра в 15, пятнашки, такен — популярная головоломка, придуманная
в 1878 году Ноем Чепмэном. Она представляет собой набор
одинаковых квадратных костяшек с нанесёнными числами, заключённых в
квадратную коробку. Длина стороны коробки в четыре раза больше длины
стороны костяшек для набора из 15 элементов, соответственно в коробке
остаётся незаполненным одно квадратное поле. Цель игры — перемещая
костяшки по коробке, добиться упорядочивания их по номерам, желательно
сделав как можно меньше перемещений.

In [6]:
from math import sqrt
from heapq import heappop, heappush

def a_star(start_chain, goal_node):
    node_hash = {} #словарь для записи передвижений каждого квадрата
    queue = [] #очередь квадратов
    heappush(queue, start_chain) #пушим в очередь
    while queue: #цикл пока очередь не пуста
        cur_chain = heappop(queue) #берем квадрат из очереди
        cur_node = cur_chain.last_node() #получаем ситуацию на доске в строке
        if cur_node == goal_node: #если ситуация на доске равна цели, заканчиваем цикл, мы решили пятнашки
            return cur_chain
        node_hash[cur_node] = cur_chain.g() #записываем длину шагов для этого квадрата
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(queue, chain)

#нахождение манхэттенского расстояния между а и b в матрице размера n
def manh_dst_matrix(a, b, n):
    return abs(a % n - b % n) + abs(a // n - b // n)

#класс пятнашек
class barley_break:

    #инициализация

    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int(sqrt(len(board_state)))
        self.history = history
        self.quad_size = int(self.size * self.size)

    #итоговое расстояние
    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += manh_dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)

    def last_node(self):
        #должен являться хеширующемся значением. Список не хешируем.
        return str(self.board_state)


    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2

    #эвристика
    def h(self):
        return self.manh_dst() + self.last_move()

    #длина истории
    def g(self):
        return len(self.history)

    #сумма эвристики и длины истории
    def f(self):
        return self.h() + self.g()

    def __lt__(self, other):
        return self.f() < other.f()

    def get_neighbours(self):
        neighbours = []
        zero_coord = self.board_state.index(0)
        if zero_coord + 1 < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighbours.append(barley_break(new_state, self.history + [self]))


        if zero_coord - 1 >= 0 and manh_dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighbours.append(barley_break(new_state, self.history + [self]))


        if zero_coord + self.size < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighbours.append(barley_break(new_state, self.history + [self]))


        if zero_coord - self.size >= 0 and manh_dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighbours.append(barley_break(new_state, self.history + [self]))
        return neighbours


if __name__ == '__main__':
    steps = []
    start_position = list(map(int, input("Введите начальное положение: ").split(" ")))
    #проверека на решаемость пятнашек
    summ = 0
    for i in range(16):
        current_number = start_position[i]
        for j in range(i+1, 16):
            if start_position[j] < current_number and start_position[j] != 0:
                summ += 1
    e = start_position.index(0) // 4 + 1
    summ += e
    if summ % 2 == 1:
        print("Данная комбинация является нерешаемой")
    else:
        start = barley_break(start_position)
        end = barley_break((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0))
        result = a_star(start, end.last_node())
        if len(list(result.history)) == 0:
            print("Пятнашки решены")
        else:
            first_node = result.history[0]
            steps = []
            for node in result.history:
                for j in range(len(node.board_state)):
                    if first_node.board_state[j] != node.board_state[j] and node.board_state[j] != 0:
                        steps.append(node.board_state[j])
                first_node = node
            if steps[len(steps) - 1] == 14:
                steps.append(15)
            else:
                steps.append(12)
            print(steps)

Введите начальное положение: 1 2 3 4 5 6 7 8 13 9 11 12 10 14 15 0
[15, 14, 10, 13, 9, 10, 14, 15]


### Вывод:

В данной лабораторной работе я реализовала два метода поиска подстроки в строке: алгоритм Кнута-Морриса-Пратта и упрощенный алгоритм Бойера-Мура, и сравнила их время работы. Также, была реализована программа, имитирующая игру в пятнашки через алгоритм A-Star.